In [ ]:
import pandas
import numpy as np
from tqdm import tqdm
import time
import csv
from matplotlib import pyplot as plt

from keras.preprocessing.image import ImageDataGenerator

#Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

# Tensorflow
import tensorflow as tf
from keras.utils.np_utils import to_categorical
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam ,RMSprop, SGD
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D,LeakyReLU

In [ ]:
SUBMISSION_HEADER = ['Id', 'class']
RANDOM_STATE = 0

In [ ]:
def open_pickled_file(file):
    with open(file, 'rb') as f:
        data = pandas.read_pickle(file)
    return np.asarray(data)

In [ ]:
def export_predictions(y_pred):
    timestr = time.strftime("%Y%m%d-%H%M%S")
    filename = '../prediction-{}.csv'.format(timestr)
    file = open(filename, 'w+', newline ='')
    with file:    
        writer = csv.writer(file)
        writer.writerow(SUBMISSION_HEADER)
        writer.writerows(enumerate(y_pred))
    print('Predictions exported to prediction.csv')

In [ ]:
def show_image(data):
    plt.imshow(data, interpolation='nearest')
    plt.show()

In [ ]:
def split_dataset(X_dataset, y_dataset, test_size=0.2, random_state=0, shuffle=False):
    return train_test_split(X_dataset, y_dataset, test_size=test_size, random_state=random_state, shuffle=shuffle)

In [ ]:
%cd ~/git/map-kaggle/

In [ ]:
X_train_raw = open_pickled_file('./dataset/x_train.pkl.zip')
y_train_raw = open_pickled_file('./dataset/y_train.pkl')
X_test_raw = open_pickled_file('./dataset/x_test.pkl.zip')

In [ ]:
print(X_train_raw.shape)
print(y_train_raw.shape)
print(X_test_raw.shape)
print(np.unique(y_train_raw))

In [ ]:
def flatten(dataframe):
    return dataframe.reshape(dataframe.shape[0], dataframe.shape[1] * dataframe.shape[2])

In [ ]:
def encode(y):
    le = preprocessing.LabelEncoder()
    le.fit(y)
    return le.transform(y)

In [ ]:
def data_augmentation(X, y, rotation_range=90):
    img = ImageDataGenerator(rotation_range = rotation_range)
    train_batch = img.flow(X, y, batch_size = 128)
    return train_batch

In [ ]:
def predict_and_export(clf, X):
    print('Predicting..')
    y_pred = clf.predict(X)
    print('Exporting predictions..')
    export_predictions(y_pred)
    print('Export completed')    

In [ ]:
# baseline - SVM
def train_svm(X_train, y_train, random_state=RANDOM_STATE, verbosity=True):
    clf = SVC(random_state=random_state, verbose=verbosity)
    clf.fit(X_train_flattened, y_train)
    return clf

# baseline - logistic regression
def train_logistic_regression(X_train, y_train, random_state=RANDOM_STATE, max_iter=100, verbosity=1):
    hyperparameters = []
    clf = LogisticRegression(max_iter=max_iter, verbose=verbosity, random_state=random_state)
    clf.fit(X_train, y_train)
    return clf

# CNN
def create_cnn(num_classes):
    model = Sequential([
        
        layers.Conv2D(32, kernel_size=(3, 3), activation='linear', input_shape=(96,96), padding='same'),
        layers.LeakyReLU(alpha=0.1),
        layers.MaxPooling2D((2, 2), padding='same'),

        layers.Conv2D(64, (3, 3), activation='linear', padding='same'),
        layers.LeakyReLU(alpha=0.1),
        layers.MaxPooling2D(pool_size=(2, 2), padding='same'),

        layers.Conv2D(128, (3, 3), activation='linear', padding='same'),
        layers.LeakyReLU(alpha=0.1),          
        layers.MaxPooling2D(pool_size=(2, 2), padding='same'),

        layers.Conv2D(256, (3, 3), activation='linear', padding='same'),
        layers.LeakyReLU(alpha=0.1),                 
        layers.MaxPooling2D(pool_size=(2, 2), padding='same'),
        layers.Dropout(0.5),

        layers.Flatten(),

        layers.Dense(128, activation='linear'),
        layers.LeakyReLU(alpha=0.1),  
        layers.Dropout(0.5),

        layers.Dense(num_classes, activation='softmax')
    ])
    return model

In [ ]:
#scale and flatten X
X_train_scaled = X_train_raw/255
X_train_scaled_reshaped = X_train_scaled.reshape(-1,96,96,1)
# X_train_scaled_flattened = flatten(X_train_scaled)

X_test_scaled = X_test_raw/255
X_test_scaled_reshaped = X_test_scaled.reshape(-1,96,96,1)

# X_test_scaled_flattened = flatten(X_test_scaled)

#encode y_train
y_train_encoded = encode(y_train_raw)
Y_train_encoded_categorical = to_categorical(y_train_encoded, 11)
X_train, X_val, y_train, y_val = split_dataset(X_train_scaled_reshaped, Y_train_encoded_categorical)

In [ ]:
# print('Training Logistic regression')
# clf = train_logistic_regression(X_train, y_train)
# predict_and_export(clf, X_test_flattened)

# print('Training SVM')
# clf = train_svm(X_train, y_train)
# predict_and_export(clf, X_test_flattened)

In [ ]:
print('Training CNN')
tf.test.is_built_with_cuda()
tf.test.gpu_device_name()

model = create_cnn(num_classes = 11)
dataload = data_augmentation(X_train, y_train,)
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.00001, patience=10)

with tf.device('/GPU:0'):
    model.compile(loss="categorical_crossentropy", optimizer=RMSprop(learning_rate=0.001), metrics=["accuracy"])
    model.fit(dataload, validation_data=(X_val, y_val), batch_size=32, epochs=100, shuffle=True, callbacks=[callback])

In [ ]:
print(model.evaluate(X_val, y_val))
pred = model.predict(X_test_scaled_reshaped)
pred = np.argmax(pred, axis=1)
export_predictions(pred)

In [ ]:
[0, 0, 0, 0.43, 0.54, 0.03...]